# HW4

### Егоров Борис

выровнял сырые риды

`bwa index MG1655-K12.first10K.fasta`

`bwa index MG1655-K12.first400K.fasta`

`bwa mem -t 4 MG1655-K12.first10K.fasta ecoli_10K_err_1.fastq ecoli_10K_err_2.fastq > test.sam`

`bwa mem -t 4 MG1655-K12.first400K.fasta ecoli_400K_err_1.fastq ecoli_400K_err_2.fastq > ecoli.sam`

`samtools view -S -b test.sam > test.bam`

`samtools view -S -b ecoli.sam > ecoli.bam`

`samtools sort test.bam -o test.sorted.bam`

`samtools sort ecoli.bam -o ecoli.sorted.bam`

`samtools index -@ 4 -b test.sorted.bam`

`samtools index -@ 4 -b ecoli.sorted.bam`

отфильтровал риды с помощью trimmomatic

`java -jar ../Trimmomatic-0.39/trimmomatic-0.39.jar PE ecoli_10K_err_1.fastq ecoli_10K_err_2.fastq -baseout test_trimmed LEADING:20 TRAILING:20 SLIDINGWINDOW:10:20 MINLEN:20`

`java -jar ../Trimmomatic-0.39/trimmomatic-0.39.jar PE ecoli_400K_err_1.fastq ecoli_400K_err_2.fastq -baseout ecoli_trimmed LEADING:20 TRAILING:20 SLIDINGWINDOW:10:20 MINLEN:20`

выровнял риды, отфильтрованные с помощью trimmomatic

`bwa mem -t 4 MG1655-K12.first10K.fasta test_trimmed_1P test_trimmed_2P > test_trimmed.sam`

`bwa mem -t 4 MG1655-K12.first400K.fasta ecoli_trimmed_1P ecoli_trimmed_2P > ecoli_trimmed.sam`

`samtools view -S -b test_trimmed.sam > test_trimmed.bam`

`samtools view -S -b ecoli_trimmed.sam > ecoli_trimmed.bam`

`samtools sort test_trimmed.bam -o test_trimmed.sorted.bam`

`samtools sort ecoli_trimmed.bam -o ecoli_trimmed.sorted.bam`

`samtools index -@ 4 -b test_trimmed.sorted.bam`

`samtools index -@ 4 -b ecoli_trimmed.sorted.bam`

исправил риды с помощью spades

`spades.py --only-error-correction --pe1-1 ecoli_10K_err_1.fastq --pe1-2 ecoli_10K_err_2.fastq -o test_spades`

`spades.py --only-error-correction --pe1-1 ecoli_400K_err_1.fastq --pe1-2 ecoli_400K_err_2.fastq -o ecoli_spades`

выровнял риды, исправленные с помощью spades

`bwa mem -t 4 MG1655-K12.first10K.fasta test_spades/corrected/ecoli_10K_err_1.00.0_0.cor.fastq.gz test_spades/corrected/ecoli_10K_err_2.00.0_0.cor.fastq.gz > test_spades.sam`

`bwa mem -t 4 MG1655-K12.first400K.fasta ecoli_spades/corrected/ecoli_400K_err_1.00.0_0.cor.fastq.gz ecoli_spades/corrected/ecoli_400K_err_2.00.0_0.cor.fastq.gz > ecoli_spades.sam`

`samtools view -S -b test_spades.sam > test_spades.bam`

`samtools view -S -b ecoli_spades.sam > ecoli_spades.bam`

`samtools sort test_spades.bam -o test_spades.sorted.bam`

`samtools sort ecoli_spades.bam -o ecoli_spades.sorted.bam`

`samtools index -@ 4 -b test_spades.sorted.bam`

`samtools index -@ 4 -b ecoli_spades.sorted.bam`

In [2]:
import pysam
import numpy as np

def stats(path_raw, path_corrected, path_ref):
    fa = pysam.FastaFile(path_ref)
    ref = fa.fetch('gi|49175990|ref|NC_000913.2|')
    fa.close()
    counts = {
        'undetected': 0,
        'detected_corrected': 0,
        'detected_removed': 0,
        'falsely_corrected': 0,
        'correctly_unmodified': 0,
        'incorrectly_removed': 0,
    }
    raw = pysam.AlignmentFile(path_raw, "rb")
    cor = pysam.AlignmentFile(path_corrected, "rb")
    i_raw = iter(raw.pileup())
    i_cor = iter(cor.pileup())
    for base_ref in ref:
        reads_raw = {}
        for read in next(i_raw).pileups:
            reads_raw[str(read.alignment).split()[0]] = read
        for read in next(i_cor).pileups:
            id = str(read.alignment).split()[0]
            if id not in reads_raw or read.alignment.is_reverse != reads_raw[id].alignment.is_reverse:
                continue
            raw_read = reads_raw[id]
            del reads_raw[id]
            base_cor = '' if read.query_position is None else read.alignment.query_sequence[read.query_position]
            base_raw = '' if raw_read.query_position is None else raw_read.alignment.query_sequence[raw_read.query_position]
            if base_raw == base_ref:
                if read.is_del == 1:
                    counts['incorrectly_removed'] += 1
                    continue
                if base_cor == base_ref:
                    counts['correctly_unmodified'] += 1
                else:
                    counts['falsely_corrected'] += 1
            else:
                if read.is_del == 1:
                    counts['detected_removed'] += 1
                    continue
                if base_cor == base_ref:
                    counts['detected_corrected'] += 1
                else:
                    counts['undetected'] += 1
        for id in reads_raw:
            if reads_raw[id].query_position is not None and base_ref == reads_raw[id].alignment.query_sequence[reads_raw[id].query_position]:
                counts['incorrectly_removed'] += 1
            else:
                counts['detected_removed'] += 1
    raw.close()
    cor.close()
    return counts

# %time print(stats('test.sorted.bam', 'test_trimmed.sorted.bam', 'MG1655-K12.first10K.fasta'))

In [120]:
%time print(stats('test.sorted.bam', 'test_spades.sorted.bam', 'MG1655-K12.first10K.fasta'))

{'undetected': 3566, 'detected_corrected': 18781, 'detected_removed': 32879, 'falsely_corrected': 7881, 'correctly_unmodified': 5217754, 'incorrectly_removed': 439746}
CPU times: user 5min 41s, sys: 179 ms, total: 5min 41s
Wall time: 5min 41s


In [3]:
print(stats('ecoli.sorted.bam', 'ecoli_trimmed.sorted.bam', 'MG1655-K12.first400K.fasta'))

{'undetected': 885231, 'detected_corrected': 657, 'detected_removed': 1538193, 'falsely_corrected': 195, 'correctly_unmodified': 240912298, 'incorrectly_removed': 26068555}


In [4]:
print(stats('ecoli.sorted.bam', 'ecoli_spades.sorted.bam', 'MG1655-K12.first400K.fasta'))

{'undetected': 246557, 'detected_corrected': 831067, 'detected_removed': 1346457, 'falsely_corrected': 372863, 'correctly_unmodified': 246720028, 'incorrectly_removed': 19888157}


| Reads | Tool        | Undetected | Detected & corrected | Detected & removed | Falsely corrected | Correctly unmodified | Incorrectly removed |
|:-----:|:-----------:| :---------:|:--------------------:|:------------------:|:-----------------:|:-----------------------:|:-------------------:|
| test  | trimmomatic |17921|10|37295|2|5092438|572941|
| test  | spades |3566|18781|32879|7881|5217754|439746|
| ecoli | trimmomatic |||||||
| ecoli | spades |||||||